# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import sys
import time
import pandas as pd
import numpy as np
import nltk 
from sqlalchemy import create_engine
from nltk.tokenize import RegexpTokenizer, wordpunct_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('MyDisasterResponse', con = engine)
df.head(2)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0


In [3]:
## To find the output variable that has more than 2 categories.
L = []
for col in df.iloc[:,4:].columns:
    if len(df[col].unique())>2:
        L.append(col)
print(L)       

['related']


In [4]:
df.related.unique()

array(['1', '0', '2'], dtype=object)

In [5]:
df = df[df.related != '2']

In [6]:
df.related.unique()

array(['1', '0'], dtype=object)

In [7]:
# change object type to int type
df.iloc[:,4:] = df.iloc[:,4:].astype(str).astype(int)

In [8]:
df.iloc[:,4:].dtypes

related                   int64
request                   int64
offer                     int64
aid_related               int64
medical_help              int64
medical_products          int64
search_and_rescue         int64
security                  int64
military                  int64
child_alone               int64
water                     int64
food                      int64
shelter                   int64
clothing                  int64
money                     int64
missing_people            int64
refugees                  int64
death                     int64
other_aid                 int64
infrastructure_related    int64
transport                 int64
buildings                 int64
electricity               int64
tools                     int64
hospitals                 int64
shops                     int64
aid_centers               int64
other_infrastructure      int64
weather_related           int64
floods                    int64
storm                     int64
fire    

In [9]:
# Create input and output variables
X = df['message'].values
Y = df.iloc[:,4:].values

In [10]:
X

array(['Weather update - a cold front from Cuba that could pass over Haiti',
       'Is the Hurricane over or is it not over',
       'Looking for someone but no name', ...,
       "Proshika, operating in Cox's Bazar municipality and 5 other unions, Ramu and Chokoria, assessment, 5 kg rice, 1,5 kg lentils to 700 families.",
       'Some 2,000 women protesting against the conduct of the elections were teargassed as they tried to converge on the local electoral commission offices in the southern oil city of Port Harcourt.',
       'A radical shift in thinking came about as a result of this meeting, recognizing that HIV/AIDS is at the core of the humanitarian crisis and identifying the crisis itself as a function of the HIV/AIDS pandemic.'], dtype=object)

In [11]:
Y

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ..., 
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

### 2. Write a tokenization function to process your text data

In [12]:
def tokenize(text):
    tokenizer = RegexpTokenizer(r'[a-zA-Z0-9]+')
    token = tokenizer.tokenize(text.lower())
    token = [item for item in token if item not in stopwords.words("english")]
    token = [WordNetLemmatizer().lemmatize(item, pos ='v') for item in token]
    return token

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [13]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf',TfidfTransformer()),
    ('clf', RandomForestClassifier()) 
])

pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7efd722f57b8>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None, verbose=0,
               warm_

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.25 )

In [ ]:
pipeline.fit(X_train, y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [16]:
y_pred = pipeline.predict(X_test)

In [17]:
print(type(y_test))
print(type(y_pred))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [23]:
labels = np.unique(y_pred)
print(labels)
#print(confusion_matrix(y_test, y_pred, labels = labels))
print(classification_report(y_test, y_pred ))

[ 0.  1.]
             precision    recall  f1-score   support

          0       0.86      0.90      0.88      4979
          1       0.82      0.46      0.59      1096
          2       0.00      0.00      0.00        36
          3       0.78      0.49      0.60      2715
          4       0.63      0.05      0.09       547
          5       0.67      0.04      0.08       312
          6       0.75      0.03      0.06       178
          7       0.00      0.00      0.00       107
          8       0.60      0.04      0.08       208
          9       0.00      0.00      0.00         0
         10       0.92      0.24      0.39       413
         11       0.86      0.45      0.59       754
         12       0.81      0.24      0.37       603
         13       0.81      0.18      0.29        96
         14       0.60      0.02      0.04       142
         15       0.00      0.00      0.00        64
         16       0.50      0.02      0.05       210
         17       0.91      0.07   

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [15]:
parameters = {
    'clf__n_estimators' : [50,100]
    #'clf__criterion' : ['gini']
}

cv = GridSearchCV(pipeline, param_grid = parameters )

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [16]:
time_start = time.time()
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)
time_end = time.time()

accuracy = (y_pred == y_test).mean()
print(classification_report(y_test, y_pred))
print('model accuracy is {}'.format(accuracy))
print('best parameter: ', cv.best_params_)
print ('time run is {}'.format((time_end - time_start)/60))

             precision    recall  f1-score   support

          0       0.85      0.93      0.89      4987
          1       0.84      0.48      0.61      1143
          2       0.00      0.00      0.00        28
          3       0.81      0.54      0.65      2780
          4       0.67      0.02      0.03       522
          5       0.67      0.02      0.04       311
          6       0.60      0.03      0.06       206
          7       0.00      0.00      0.00       119
          8       0.56      0.04      0.08       217
          9       0.00      0.00      0.00         0
         10       0.89      0.25      0.39       411
         11       0.85      0.41      0.55       735
         12       0.85      0.25      0.38       596
         13       0.87      0.11      0.19       120
         14       0.57      0.02      0.05       169
         15       1.00      0.01      0.02        92
         16       0.20      0.01      0.01       191
         17       0.78      0.10      0.17   

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.